<a href="https://colab.research.google.com/github/efearase/RL_with_sentiment/blob/main/text_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
from transformers import pipeline
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output
tokenizer = RegexTokenizer()
tokens = tokenizer.split('всё очень плохо')

model = FastTextSocialNetworkModel(tokenizer=tokenizer)

classifier = pipeline('sentiment-analysis', model='blanchefort/rubert-base-cased-sentiment')

In [2]:
def filter_text(input_text):
    allowed_chars = set(
        'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
        '0123456789 .,!?:;'
        'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
        'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
    )
    return ''.join(c for c in input_text if c in allowed_chars)

def select_message(text, name='CBOM'):
    arr = text.split(".")
    return ''.join(sent for sent in arr if name in sent)

In [3]:
tokenizer = RegexTokenizer()
tokens = tokenizer.split('всё очень плохо')  # [('всё', None), ('очень', None), ('плохо', None)]

model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [4]:
def sentiment(df):
    sentiment_class = []
    sentiment_score = []
    classifier = pipeline('sentiment-analysis', model='blanchefort/rubert-base-cased-sentiment')
    t_0 = 0
    print('Start fancy...')
    batch_results = []  # Initialize batch_results to an empty list
    for i in range(len(df)):
        if i - t_0 > 100:
            print(f"progress: {round(i/len(df)*100, 2)}%")
            clear_output(wait=True)
            t_0 = i
        try:
            res = classifier(df['text'][i])
        except Exception as e:
            try:
                # If the initial classification fails, try with a truncated version of the text
                res = classifier(df['text'][i][:512])
            except Exception as e:
                # If it still fails, skip this record or use a default value
                print(f"Error processing text at index {i}: {e}")
                res = [{'label': 'neutral', 'score': 0}]  # Example default value
        batch_results.extend(res)

    sentiment_class = [res['label'] for res in batch_results]
    sentiment_score = [res['score'] for res in batch_results]

    df['sentiment_class'] = sentiment_class
    df['sentiment_score'] = sentiment_score

    dostoevski_label = []
    dostoevski_score = []
    
    results = model.predict(df['text'], k=1)

    for result in results:
        dostoevski_label.append(list(result.keys())[0])
        dostoevski_score.append(list(result.values())[0])

    df['dostoevski_class'] = dostoevski_label
    df['dostoevski_score'] = dostoevski_score

    replacement_dict = {'positive': 1, 'negative': -1, 'neutral': 0,
                       'POSITIVE': 1, 'NEGATIVE': -1, 'NEUTRAL': 0, 'speech' : 0, 'skip': 0}

    df.replace(replacement_dict, inplace=True)

    df['bert_score'] = df['sentiment_class']*df['sentiment_score']
    df['dostoevski_score'] = df['dostoevski_class']*df['dostoevski_score']

    df.drop(columns=['sentiment_class', 'sentiment_score', 'dostoevski_class'],
            inplace =True)

    return df

In [8]:
for file in os.listdir('data'):
    print(file)
    if file.endswith('csv') and file not in os.listdir('text_data'):
        ticker = file.split("_")[1].split(".")[0]
        def select_message(text, name=ticker):
            arr = text.split(".")
            return ''.join(sent for sent in arr if name in sent)
        df = pd.read_csv("data/" + file, index_col=0)
        try:
            df["text"].apply(filter_text)
            df["text"].apply(select_message)
            print('getting sentiment...')
            df_new = pd.DataFrame()
            df_new = sentiment(df)
            df_new.to_csv("text_data/" + file)
        except TypeError:
            pass

progress: 99.96%
